In [29]:
import cv2 
import numpy as np 
from matplotlib import pyplot as plt
from pathlib import Path

In [ ]:
#tableaux contenant les logos et images du jeu de données car on ne peut pas parcourir simplement un dossier avec opencv

jeuDeDonneesLogo = ["Bershka-logo.png","primark-logo.png","zara-logo.jpg","hm-logo.png","Shein-logo.png"]
jeuDeDonneesImage = ["BERSHKA.jpg","PRIMARK.jpg","ZARA.jpeg","H&M.jpg","SHEIN.jpeg"]

def afficher_logos(): #permet d'afficher les logos pour nous permettre de les tester au besoin
    for i in jeuDeDonneesLogo:
        fichier = i
        img = cv2.imread(fichier, cv2.IMREAD_COLOR) 

        cv2.imshow("Titre", img) 
        cv2.waitKey(0) 
        cv2.destroyAllWindows()

def afficher_images(): #permet d'afficher les images pour nous permettre de les tester au besoin
    for i in jeuDeDonneesImage:
        fichier = i
        img = cv2.imread(fichier, cv2.IMREAD_COLOR) 

        cv2.imshow("Titre", img) 
        cv2.waitKey(0) 
        cv2.destroyAllWindows()

def afficher_image(fichier): #permet d'afficher une image spécifique
    img = cv2.imread(fichier, cv2.IMREAD_COLOR) 
    cv2.imshow("Titre", img) 
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

def resize_image(fichier, largeur, hauteur): #permet de redimensionner une image spécifique
    resized_image = cv2.resize(fichier, (largeur, hauteur))
    return resized_image

In [31]:
#extensions acceptées
ext=['.png', '.jpg', '.jpeg']


def main_for_user():
    #Demande du nom de l'image à l'utilisateur
    nom_fichier = str(input("Entrez le nom de l'image (avec ou sans extension): "))

    #vérification de l'extension si elle est donnée
    if (not nom_fichier.lower()[-3:] in ext) or (not nom_fichier.lower()[-4:] in ext):
        new_ext = str(input("Entrez l'extension du fichier (avec ou sans le point): "))

        #vérification que l'extension commence bien par un point

        if new_ext[0] != '.':
            new_ext = '.' + new_ext
        nom_fichier += new_ext

        afficher_image(nom_fichier)

In [ ]:
#algothime de recherche des logos dans les images du jeu de données
#pour les tests

def main_for_test():
    for i in range(len(jeuDeDonneesImage)):
        imgLogo = cv2.imread(jeuDeDonneesLogo[i],cv2.IMREAD_GRAYSCALE)
        img = cv2.imread(jeuDeDonneesImage[i],cv2.IMREAD_GRAYSCALE)

        # Extraction des caractéristiques ORB des images.
        orb = cv2.ORB_create()
        kp0, des0 = orb.detectAndCompute(imgLogo, None)
        kp1, des1 = orb.detectAndCompute(img, None)

        # Matching par force brute (méthode de plus proches voisins)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = bf.match(des0, des1)

        # Tri des paires des matches par distance.
        matches = sorted(matches, key=lambda x:x.distance)

        # Affichage de 25 meilleurs matches.
        img_matches = cv2.drawMatches(
            imgLogo, kp0, img, kp1, matches[:25], img,
            flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
        
        # Affichage de l'image.
        cv2.imshow("Titre", resize_image(img_matches, 800, 600))
        cv2.waitKey(0) 
        cv2.destroyAllWindows()
    

In [33]:
main_for_test()